In [1]:
import os
from datetime import datetime
from calendar import monthrange
import polars as pl
import altair as alt

In [2]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
pl.Config(tbl_rows=200)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [5]:
df = pl.read_parquet(
    os.path.join("data","fdb.parquet")
).with_columns(
    pl.col("datum").dt.month().alias("mesic")
).with_columns(
    pl.col("datum").dt.week().alias("tyden")
).with_columns(
    pl.col("datum").dt.year().alias("rok")
).with_columns(
    pl.when(
        pl.col("url").str.contains("serial/")
    ).then(
        pl.lit("serial")
    ).when(
        pl.col("url").str.contains("film/")
    ).then(
        pl.lit("film")
    ).alias(
        "kategorie"
    )
).sort(
    by="datum"
).with_columns(
    pl.col("nazev").replace({'Šimon a Matouš jedou na rivieru':'Šimon a Matouš jedou na Riviéru'})
)

In [6]:
kanaly = df.select(pl.col("kanal")).unique().to_series().to_list()
kanaly

['Nova Action',
 'ČT2',
 'Film Europe Channel',
 'Prima Show',
 'Cinemax',
 'Prima',
 'CS Film',
 'Nova Lady',
 'Film+',
 'Nova 2',
 'Cinemax2',
 'NOVA',
 'ČT1',
 'Prima LOVE',
 'Prima STAR',
 'Prima Cool',
 'TV Barrandov',
 'Nova Cinema',
 'ČT 3',
 'VOYO',
 'CNN Prima News',
 'Nova Fun']

In [7]:
verejnopravni = [k for k in kanaly if 'ČT' in k]
print(verejnopravni)
soukrome = [k for k in kanaly if 'ČT' not in k]
print(soukrome)

['ČT2', 'ČT1', 'ČT 3']
['Nova Action', 'Film Europe Channel', 'Prima Show', 'Cinemax', 'Prima', 'CS Film', 'Nova Lady', 'Film+', 'Nova 2', 'Cinemax2', 'NOVA', 'Prima LOVE', 'Prima STAR', 'Prima Cool', 'TV Barrandov', 'Nova Cinema', 'VOYO', 'CNN Prima News', 'Nova Fun']


In [8]:
vysilane_verejnopravni = df.filter(pl.col("kanal").is_in(verejnopravni)).select(pl.col("url")).unique().to_series().to_list()
vysilane_soukromymi = df.filter(pl.col("kanal").is_in(soukrome)).select(pl.col("url")).unique().to_series().to_list()
vysilane_vsude = [x for x in vysilane_verejnopravni if x in vysilane_soukromymi]

In [9]:
aspon_na_dvou = df.group_by("url").agg(pl.col("kanal").unique().len()).filter(pl.col("kanal") >= 2).select(pl.col("url")).to_series().to_list()

In [10]:
zajima_nas = ['ČT1', 'ČT2', 'NOVA', 'Prima']

In [11]:
v_poslednich_dvou_letech = df.filter(pl.col("kanal").is_in(zajima_nas)).filter(pl.col("datum").dt.year().is_in([2023,2024])).filter(pl.col("datum").dt.month().is_in([7,8])).select(pl.col("url")).unique().to_series().to_list()

In [12]:
x_let = df.with_columns(pl.col('datum').dt.year().alias('rok')).group_by('url').agg(pl.col('rok').unique().len()).filter(pl.col('rok') >= 5).select(pl.col('url')).unique().to_series().to_list()

In [13]:
vsude_a_x_let_pracovni = [x for x in x_let if x in aspon_na_dvou]
vsude_a_x_let = [x for x in vsude_a_x_let_pracovni if x in v_poslednich_dvou_letech]

In [14]:
letni = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('url').is_in(vsude_a_x_let) & pl.col('mesic').is_in([7,8])).group_by('url').len().rename({'len':'v_lete'})

In [15]:
zimni = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('url').is_in(vsude_a_x_let) & (~pl.col('mesic').is_in([6,7,8,9]))).group_by('url').len().rename({'len':'v_zime'})

In [16]:
pomer = letni.join(zimni, how='full', on='url').with_columns(
    pl.when(
        pl.col('url').is_null()
    ).then(
        pl.col('url_right')
    ).otherwise(
        pl.col('url')
    ).alias(
        'url'
    )
).with_columns(
    pl.col('v_lete').fill_null(0)
).drop(
    'url_right'
).with_columns(
    (pl.col('v_lete') / pl.col('v_zime')).alias('pomer')
)

In [17]:
df.filter(pl.col("nazev").str.contains("Léto s kov"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2007-12-25 13:40:00,"""Léto s kovbojem (?)""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",12,52,2007,"""film"""
"""NOVA""",2008-11-22 16:00:00,"""Léto s kovbojem (?)""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",11,47,2008,"""film"""
"""Nova Cinema""",2009-03-08 19:10:00,"""Léto s kovbojem (?)""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",3,10,2009,"""film"""
"""NOVA""",2009-11-28 15:30:00,"""Léto s kovbojem""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",11,48,2009,"""film"""
"""Nova Cinema""",2010-06-20 20:00:00,"""Léto s kovbojem""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",6,24,2010,"""film"""
"""NOVA""",2010-09-28 12:10:00,"""Léto s kovbojem""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",9,39,2010,"""film"""
"""NOVA""",2011-09-04 15:10:00,"""Léto s kovbojem""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",9,35,2011,"""film"""
"""NOVA""",2012-09-01 17:30:00,"""Léto s kovbojem""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",9,35,2012,"""film"""
"""Nova Cinema""",2013-02-02 20:00:00,"""Léto s kovbojem""","""06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""Další díl/opakování:06.10.2024 v 16:05 (NOVA)08.10.2024 v 15:05 (Nova Cinema)10.10.2024 v 12:10 (Nova Cinema)""","""film/leto-s-kovbojem/11391""",2,5,2013,"""film"""


In [18]:
df.select(pl.col(['url','nazev'])).unique().filter(pl.col('nazev').str.contains('Četník'))

url,nazev
str,str
"""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""","""Četník ze Saint Tropez (?)"""
"""film/cetnik-v-new-yorku-le-gendarme-a-new-york/5617""","""Četník v New Yorku (?)"""
"""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""","""Četník ze Saint Tropez"""
"""film/cetnik-se-zeni-le-gendarme-se-marie/5615""","""Četník se žení"""
"""film/cetnik-se-zeni-le-gendarme-se-marie/5615""","""Četník se žení (?)"""
"""film/cetnik-ve-vysluzbe-le-gendarme-en-balade/5618""","""Četník ve výslužbě (?)"""
"""film/cetnik-v-new-yorku-le-gendarme-a-new-york/5617""","""Četník v New Yorku"""
"""film/cetnik-a-cetnice-le-gendarme-et-les-gendarmettes/5613""","""Četník a četnice (?)"""
"""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""","""Četník ze St. Tropez"""


In [19]:
df.filter(pl.col("url") == "film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2008-02-06 20:00:00,"""Četník a mimozemšťané (?)""","""15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""Další díl/opakování:15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",2,6,2008,"""film"""
"""NOVA""",2008-03-22 14:50:00,"""Četník a mimozemšťané (?)""","""15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""Další díl/opakování:15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",3,12,2008,"""film"""
"""NOVA""",2009-03-29 15:10:00,"""Četník a mimozemšťané (?)""","""15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""Další díl/opakování:15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",3,13,2009,"""film"""
"""NOVA""",2009-12-30 14:45:00,"""Četník a mimozemšťané""","""15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""Další díl/opakování:15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",12,53,2009,"""film"""
"""NOVA""",2010-10-30 16:40:00,"""Četník a mimozemšťané""","""15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""Další díl/opakování:15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",10,43,2010,"""film"""
"""NOVA""",2012-01-06 17:50:00,"""Četník a mimozemšťané""","""15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""Další díl/opakování:15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",1,1,2012,"""film"""
"""Nova Cinema""",2012-01-07 14:20:00,"""Četník a mimozemšťané""","""17.02.2024 v 22:40 (ČT1)22.02.2024 v 14:50 (ČT1)""","""Další díl/opakování:17.02.2024 v 22:40 (ČT1)22.02.2024 v 14:50 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",1,1,2012,"""film"""
"""NOVA""",2013-01-12 17:40:00,"""Četník a mimozemšťané""","""15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""Další díl/opakování:15.03.2025 v 22:05 (ČT1)19.03.2025 v 22:30 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",1,2,2013,"""film"""
"""Nova Cinema""",2013-01-13 14:35:00,"""Četník a mimozemšťané""","""17.02.2024 v 22:40 (ČT1)22.02.2024 v 14:50 (ČT1)""","""Další díl/opakování:17.02.2024 v 22:40 (ČT1)22.02.2024 v 14:50 (ČT1)""","""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",1,2,2013,"""film"""


In [20]:
pomer.filter(pl.col("url") == "film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614")

url,v_lete,v_zime,pomer
str,u32,u32,f64


In [21]:
nejsou_filmy = [
    "film/klenoty-nasi-krajiny/130962",
    "film/jsou-hvezdy-ktere-nehasnou/56077"	
]

nechceme = [
    "film/uvolnete-se-prosim-uvolnete-se-prosim-aneb-co-jste-v-televizi-nevideli-i/45382"
]

In [22]:
nazvy = df.group_by('url').agg(pl.col('nazev').mode()).explode('nazev')

In [23]:
letni_televize = pomer.filter(
    ~pl.col('url').is_in(nechceme)
).filter(
    (pl.col("pomer") > 2.1) | (pl.col("pomer").is_null())
).filter(
    pl.col('url').is_not_null()
).sort(
    by='pomer',descending=True
).with_columns(
    pl.when(pl.col('url').str.contains('film/')).then(pl.lit('film')).otherwise(pl.lit('serial')).alias('kategorie')
).filter(
    ((pl.col('kategorie') == 'film') & (pl.col('v_lete') >= 5)) | ((pl.col('kategorie') == 'serial') & (pl.col('v_lete') >= 15))
).with_columns(
    pl.when(pl.col('url').is_in(nejsou_filmy)).then(pl.lit('serial')).otherwise(pl.col('kategorie')).alias('kategorie')
).join(
    nazvy, how='left', on='url'
)

razeni = letni_televize.sort(by=['pomer','v_lete','nazev'], descending=[True,True,True]).select(pl.col("nazev")).to_series().to_list()
vyber_strojovy = letni_televize.select(pl.col('url')).to_series().to_list()

letni_televize

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/indiana-jones-a-chram-zkazy-indiana-jones-and-the-temple-of-doom/9107""",13,null,null,"""film""","""Indiana Jones a Chrám zkázy"""
"""film/2bobule/52654""",14,null,null,"""film""","""2Bobule"""
"""film/indiana-jones-a-kralovstvi-kristalove-lebky-indiana-jones-and-the-kingdom-of-the-crystal-skull/9106""",13,null,null,"""film""","""Indiana Jones a Království křišťálové lebky"""
"""film/pasians/37398""",5,null,null,"""film""","""Pasiáns"""
"""film/lepsi-uz-to-nebude-as-good-as-it-gets/24961""",5,null,null,"""film""","""Lepší už to nebude"""
"""film/indiana-jones-a-posledni-krizova-vyprava-indiana-jones-and-the-last-crusade/9108""",13,null,null,"""film""","""Indiana Jones a Poslední křížová výprava"""
"""film/bobule/44499""",14,null,null,"""film""","""Bobule"""
"""film/pearl-harbor/28161""",7,null,null,"""film""","""Pearl Harbor"""
"""serial/pisen-pro-rudolfa-iii/45231""",24,null,null,"""serial""","""Píseň pro Rudolfa III."""


In [25]:
len(letni_televize)

69

In [26]:
len(letni_televize.filter(pl.col("kategorie") == "film"))

60

In [27]:
len(letni_televize.filter(pl.col("kategorie") == "serial"))

9

In [28]:
df.filter(pl.col("url") == "serial/mama-mom/125805")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2015-06-29 16:30:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",6,27,2015,"""serial"""
"""NOVA""",2015-06-30 00:55:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",6,27,2015,"""serial"""
"""NOVA""",2015-06-30 16:30:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",6,27,2015,"""serial"""
"""NOVA""",2015-07-01 00:45:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",7,27,2015,"""serial"""
"""NOVA""",2015-07-01 16:30:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",7,27,2015,"""serial"""
"""NOVA""",2015-07-02 01:10:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",7,27,2015,"""serial"""
"""NOVA""",2015-07-02 16:30:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",7,27,2015,"""serial"""
"""NOVA""",2015-07-03 00:55:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",7,27,2015,"""serial"""
"""NOVA""",2015-07-03 16:30:00,"""Máma""","""28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 07:35 (Nova Fun)03.12…","""Další díl/opakování:28.11.2024 v 07:20 (Nova Fun)29.11.2024 v 07:20 (Nova Fun)30.11.2024 v 07:20 (Nova Fun)01.12.2024 v 07:30 (Nova Fun)02.12.2024 v 0…","""serial/mama-mom/125805""",7,27,2015,"""serial"""


In [29]:
graf = df.filter(
    pl.col("rok").is_between(2015,2024)
).filter(
    pl.col('url').is_in(vyber_strojovy)
).filter(
    ~pl.col('url').is_in(nechceme)
).filter(
    pl.col("kanal").is_in(zajima_nas)
).group_by(
    ['url','tyden']
).len(
).join(
    letni_televize.select(pl.col(["nazev","url","kategorie"])), on='url', how='left'
).with_columns(
    pl.col('kategorie').replace_strict(
        {'film':'filmy','serial':'seriály a TV pořady'}
    )
)

graf

url,tyden,len,nazev,kategorie
str,i8,u32,str,str
"""film/bobule/44499""",28,1,"""Bobule""","""filmy"""
"""serial/vinari/126130""",4,1,"""Vinaři""","""seriály a TV pořady"""
"""film/vinnetou-a-misenka-apanaci-winnetou-und-das-halbblut-apanatschi/22532""",3,1,"""Vinnetou a míšenka Apanači""","""filmy"""
"""film/lepsi-uz-to-nebude-as-good-as-it-gets/24961""",32,1,"""Lepší už to nebude""","""filmy"""
"""film/angelika-a-kral-angelique-et-le-roi/3072""",29,1,"""Angelika a král""","""filmy"""
"""serial/dobra-voda/40349""",26,2,"""Dobrá Voda""","""seriály a TV pořady"""
"""film/policejni-akademie-6-mesto-v-ohrozeni-police-academy-6-city-under-slege/16317""",35,1,"""Policejní akademie 6: Město v obležení""","""filmy"""
"""film/jursky-park-3-jurassic-park-iii/28683""",28,1,"""Jurský park 3""","""filmy"""
"""film/pearl-harbor/28161""",31,1,"""Pearl Harbor""","""filmy"""


In [30]:
roky = range(2007,2025)
prazdniny = []
for r in roky:
    prazdniny.append({'zacatek':datetime(year=r,month=7,day=1), 'konec':datetime(year=r,month=8,day=31)})

prazdniny = pl.DataFrame(prazdniny).with_columns(pl.col('zacatek').dt.week()).with_columns(pl.col('konec').dt.week())
prazdniny

df_prazdniny = []
for x in range(1,54):
    df_prazdniny.append({'tyden':x, 'prazdniny':len(prazdniny.filter((pl.col('zacatek') <= x) & (pl.col('konec') >= x)))})

df_prazdniny = pl.DataFrame(df_prazdniny).with_columns(pl.col('prazdniny') / len(roky)).filter(pl.col("prazdniny") != 0).with_columns(
    pl.col("tyden").min().alias("zacatek")
).with_columns(
    pl.col("tyden").max().alias("konec")
).with_columns(
    pl.when(pl.col('prazdniny') < 1).then(pl.lit('nekdy')).otherwise(pl.lit('vzdycky')).alias('prazdninovost')
).with_columns(
    (pl.col('tyden') + 1).alias("tyden2")
).unique(
    subset=['zacatek','prazdninovost','prazdniny']
).with_columns(
    pl.when(pl.col('prazdninovost') == 'vzdycky').then(
        pl.lit(36)
    ).otherwise(
        pl.col('tyden2')
    ).alias('tyden2')
)

df_prazdniny

tyden,prazdniny,zacatek,konec,prazdninovost,tyden2
i64,f64,i64,i64,str,i64
26,0.555556,26,36,"""nekdy""",27
36,0.166667,26,36,"""nekdy""",37
27,1.0,26,36,"""vzdycky""",36


In [31]:
terminy = alt.Chart(
    df_prazdniny
).mark_rect(color='#D5B574'
).encode(
    alt.X('tyden:Q'),
    alt.X2('tyden2:Q'),
    alt.Opacity('prazdninovost:N', scale=alt.Scale(range=[0.4,0.75]), legend=None)
)

terminy

alt.Chart(...)

In [32]:
obsah = alt.Chart(
    graf,
    title=alt.Title(
        'Prázdniny 2015 až 2024 na ČT1, ČT2, Nova a Primě',
        subtitle=[
            'Filmy, seriály a pořady, které běžely alespoň na dvou kanálech, vysílaly se v posledních dvou',
            'letech, a které jsou na obrazovce víc než dvakrát častěji v červenci a srpnu než od října do května.'
        ],
        subtitleFontSize=9.3
    )
).mark_circle(
).encode(
    alt.X('tyden:Q', axis=alt.Axis(
        title="týden v roce", tickColor='#F1D1BB', tickOpacity=0.6, tickWidth = 0.5,
        labelExpr="datum.label + '.'"
    ), scale=alt.Scale(
        domainMin = 1,
        domainMax = 52,
        padding=0,        # Remove extra padding
        nice=False        # Prevent extending to "nice" numbers
    )),
    alt.Y('nazev:N', sort=razeni, axis=alt.Axis(title=None, tickColor='#F1D1BB', tickOpacity=0.6, tickWidth = 0.5)),
    alt.Size('len:Q', scale=alt.Scale(range=[25,300]), legend=None),
    alt.Color('kategorie:N', scale=alt.Scale(range=['#D15A41','#788A98']), legend=alt.Legend(title=None, orient="top", direction="horizontal"))
).properties(
    width="container"
)

popisek = alt.Chart(
    pl.DataFrame({'tyden':39})
).mark_text(
    font='Asap',
    color='#292829',
    fontSize=8,
    text=['← týdny,','do kterých','zasáhly letní','prázdniny'],
    align='left',
    dy=-438
).encode(alt.X("tyden:Q"))

finale = alt.layer(terminy, popisek, obsah).configure_view(
    stroke='transparent'
).configure_axis(grid=True, gridOpacity=0.6, gridWidth = 0.5, gridColor='#F1D1BB').properties(
    width=260,
    height=930
)

finale

alt.LayerChart(...)

In [33]:
me_to_neurazi(graf=finale, kredity="vizualizace: iROZHLAS.cz | televizní programy: fdb.cz", slozka_na_serveru="grafy-tv", soubor='letni-program')

<figure>
    <a href="https://data.irozhlas.cz/grafy-tv/letni-program.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy-tv/letni-program.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [34]:
len(letni_televize.filter(pl.col('kategorie') == 'serial'))

9

In [35]:
len(letni_televize.filter(pl.col('kategorie') == 'film'))

60

In [36]:
letni_televize.filter(pl.col('nazev').str.contains("Vinnet"))

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/old-firehand-winnetou-und-sein-freund-old-firehand/14926""",12,1,12.0,"""film""","""Vinnetou a Old Firehand"""
"""film/vinnetou-rudy-gentleman-winnetou-ii/22531""",16,2,8.0,"""film""","""Vinnetou - Rudý gentleman"""
"""film/vinnetou-winnetou-i/22529""",16,2,8.0,"""film""","""Vinnetou"""
"""film/vinnetou-posledni-vystrel-winnetou-iii/22530""",16,2,8.0,"""film""","""Vinnetou - Poslední výstřel"""
"""film/vinnetou-a-misenka-apanaci-winnetou-und-das-halbblut-apanatschi/22532""",12,2,6.0,"""film""","""Vinnetou a míšenka Apanači"""


In [37]:
letni_televize.filter(pl.col('nazev').str.contains("Jursk"))

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/jursky-park-jurassic-park/9814""",8,1,8.0,"""film""","""Jurský park"""
"""film/ztraceny-svet-jursky-park-the-lost-world-jurassic-park/24152""",8,1,8.0,"""film""","""Ztracený svět: Jurský park"""
"""film/jursky-park-3-jurassic-park-iii/28683""",8,1,8.0,"""film""","""Jurský park 3"""


In [38]:
df.filter(pl.col('url').str.contains("film/pearl-"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2007-11-09 20:00:00,"""Pearl Harbor (?)""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",11,45,2007,"""film"""
"""Prima""",2008-08-02 20:00:00,"""Pearl Harbor (?)""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",8,31,2008,"""film"""
"""Prima""",2009-09-11 20:00:00,"""Pearl Harbor (?)""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",9,37,2009,"""film"""
"""Prima Cool""",2009-12-13 20:15:00,"""Pearl Harbor""","""05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""Další díl/opakování:05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""film/pearl-harbor/28161""",12,50,2009,"""film"""
"""Prima""",2012-07-29 20:00:00,"""Pearl Harbor""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",7,30,2012,"""film"""
"""Prima LOVE""",2013-08-31 20:15:00,"""Pearl Harbor""","""05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""Další díl/opakování:05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""film/pearl-harbor/28161""",8,35,2013,"""film"""
"""Prima""",2014-07-06 21:30:00,"""Pearl Harbor""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",7,27,2014,"""film"""
"""NOVA""",2016-06-24 20:20:00,"""Pearl Harbor""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",6,25,2016,"""film"""
"""Nova Cinema""",2016-06-25 14:05:00,"""Pearl Harbor""","""05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""Další díl/opakování:05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""film/pearl-harbor/28161""",6,25,2016,"""film"""


In [39]:
letni_televize.filter(pl.col('nazev').str.contains("Angelika"))

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/angelika-a-kral-angelique-et-le-roi/3072""",10,4,2.5,"""film""","""Angelika a král"""
"""film/angelika-markyza-andelu-angelique-marquise-des-anges/3074""",10,4,2.5,"""film""","""Angelika, markýza andělů"""
"""film/bajecna-angelika-merveilleuse-angelique/3466""",10,4,2.5,"""film""","""Báječná Angelika"""
"""film/nezkrotna-angelika-l-indomptable-angelique/14236""",9,4,2.25,"""film""","""Nezkrotná Angelika"""


In [40]:
letni_televize.filter(pl.col('nazev').str.contains("Policej"))

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/policejni-akademie-3-znovu-ve-vycviku-police-academy-3-back-in-training/16314""",14,5,2.8,"""film""","""Policejní akademie 3: Znovu ve výcviku"""
"""film/policejni-akademie-5-nasazeni-v-miami-beach-police-academy-5-assignment-miami-beach/16316""",14,5,2.8,"""film""","""Policejní akademie 5: Nasazení: Miami Beach"""
"""film/policejni-akademie-4-obcanska-patrola-police-academy-4-citizens-on-patrol/16315""",14,5,2.8,"""film""","""Policejní akademie 4: Občanská patrola"""
"""film/policejni-akademie-6-mesto-v-ohrozeni-police-academy-6-city-under-slege/16317""",14,5,2.8,"""film""","""Policejní akademie 6: Město v obležení"""
"""film/policejni-akademie-police-academy/16311""",12,5,2.4,"""film""","""Policejní akademie"""
"""film/policejni-akademie-2-jejich-prvni-nasazeni-police-academy-2-their-first-assignment/16313""",14,6,2.333333,"""film""","""Policejní akademie 2: Jejich první nasazení"""
"""film/policejni-akademie-7-moskevska-mise-police-academy-mission-to-moscow/16312""",11,5,2.2,"""film""","""Policejní akademie 7: Moskevská mise"""


In [41]:
letni_televize.filter(pl.col('nazev').str.contains("Sissi"))

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/sissi-osudova-leta-cisarovny-sissi-schicksalsjahre-einer-kaiserin/18881""",11,4,2.75,"""film""","""Sissi - Osudová léta císařovny"""
"""film/sissi-mlada-cisarovna-sissi-die-junge-kaiserin/18879""",10,4,2.5,"""film""","""Sissi - Mladá císařovna"""


In [42]:
letni_televize.filter(pl.col('nazev').str.contains("Dannyho"))

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/dannyho-partaci-3-ocean-s-thirteen/42969""",8,1,8.0,"""film""","""Dannyho parťáci 3"""
"""film/dannyho-partaci-2-ocean-s-twelve/38285""",6,1,6.0,"""film""","""Dannyho parťáci 2"""
"""film/dannyho-partaci-ocean-s-eleven/29906""",5,1,5.0,"""film""","""Dannyho parťáci"""


In [43]:
letni_televize.filter(pl.col('nazev').str.contains("Blázniv"))

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/blazniva-strela-2-a-1-2-vune-strachu-the-naked-gun-2-1-2-the-smell-of-fear/4266""",6,2,3.0,"""film""","""Bláznivá střela 2 1/2: Vůně strachu"""
"""film/blazniva-strela-33-1-3-posledni-trapas-naked-gun-33-1-3-the-final-insult/4267""",5,2,2.5,"""film""","""Bláznivá střela 33 1/3: Poslední trapas"""


In [44]:
letni_televize.filter(pl.col('nazev').str.contains("India"))

url,v_lete,v_zime,pomer,kategorie,nazev
str,u32,u32,f64,str,str
"""film/indiana-jones-a-chram-zkazy-indiana-jones-and-the-temple-of-doom/9107""",13,null,null,"""film""","""Indiana Jones a Chrám zkázy"""
"""film/indiana-jones-a-kralovstvi-kristalove-lebky-indiana-jones-and-the-kingdom-of-the-crystal-skull/9106""",13,null,null,"""film""","""Indiana Jones a Království křišťálové lebky"""
"""film/indiana-jones-a-posledni-krizova-vyprava-indiana-jones-and-the-last-crusade/9108""",13,null,null,"""film""","""Indiana Jones a Poslední křížová výprava"""
"""film/indiana-jones-a-dobyvatele-ztracene-archy-raiders-of-the-lost-ark/6614""",13,null,null,"""film""","""Indiana Jones a dobyvatelé ztracené archy"""


In [45]:
df.filter(pl.col("nazev") == "Pearl Harbor")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Prima Cool""",2009-12-13 20:15:00,"""Pearl Harbor""","""05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""Další díl/opakování:05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""film/pearl-harbor/28161""",12,50,2009,"""film"""
"""ČT2""",2010-04-07 15:05:00,"""Pearl Harbor""",null,"""Pomocí nejmodernější počítačové animace můžete sledovat vojenské operace slavné letadlové lodi od počátku války v Pacifiku až do porážky Japonského cí…",null,4,14,2010,null
"""Prima""",2012-07-29 20:00:00,"""Pearl Harbor""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",7,30,2012,"""film"""
"""Prima LOVE""",2013-08-31 20:15:00,"""Pearl Harbor""","""05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""Další díl/opakování:05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""film/pearl-harbor/28161""",8,35,2013,"""film"""
"""Prima""",2014-07-06 21:30:00,"""Pearl Harbor""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",7,27,2014,"""film"""
"""NOVA""",2016-06-24 20:20:00,"""Pearl Harbor""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",6,25,2016,"""film"""
"""Nova Cinema""",2016-06-25 14:05:00,"""Pearl Harbor""","""05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""Další díl/opakování:05.07.2024 v 13:50 (NOVA)08.07.2024 v 20:00 (Nova Cinema)09.07.2024 v 13:00 (Nova Cinema)""","""film/pearl-harbor/28161""",6,25,2016,"""film"""
"""NOVA""",2017-07-06 15:25:00,"""Pearl Harbor""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",7,27,2017,"""film"""
"""NOVA""",2017-07-07 00:20:00,"""Pearl Harbor""","""23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""Další díl/opakování:23.02.2025 v 20:00 (Prima MAX)25.02.2025 v 00:10 (Prima MAX)02.03.2025 v 09:40 (Prima MAX)""","""film/pearl-harbor/28161""",7,27,2017,"""film"""


In [46]:
df.filter(pl.col("nazev").str.contains("Chalupáři")).group_by_dynamic(index_column="datum", every="1y").agg(pl.col("noticka").len())

datum,noticka
datetime[μs],u32
2008-01-01 00:00:00,11
2009-01-01 00:00:00,11
2010-01-01 00:00:00,11
2012-01-01 00:00:00,22
2013-01-01 00:00:00,22
2014-01-01 00:00:00,22
2015-01-01 00:00:00,23
2016-01-01 00:00:00,44
2017-01-01 00:00:00,11


## Poměr komplet

In [48]:
letni_full = df.filter(pl.col("kanal").is_in(zajima_nas)).filter(pl.col('mesic').is_in([7,8])).group_by('url').len().rename({'len':'v_lete'})
zimni_full = df.filter(pl.col("kanal").is_in(zajima_nas)).filter(pl.col('mesic').is_in([1,2,3,4,5,10,11,12])).group_by('url').len().rename({'len':'v_zime'})

In [49]:
pomer_full = letni_full.join(zimni_full, how='full', on='url').with_columns(
    pl.when(
        pl.col('url').is_null()
    ).then(
        pl.col('url_right')
    ).otherwise(
        pl.col('url')
    ).alias(
        'url'
    )
).with_columns(
    pl.col('v_lete').fill_null(0)
).drop(
    'url_right'
).with_columns(
    (pl.col('v_lete') / pl.col('v_zime')).alias('pomer')
).join(
    df.group_by("url").agg(pl.col("nazev").mode()).explode("nazev"), on="url", how="left"
)

In [50]:
pomer_full.filter(pl.col("url") == "film/leto-s-kovbojem/11391")

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/leto-s-kovbojem/11391""",1,24,0.041667,"""Léto s kovbojem"""


In [51]:
pomer_full.filter(pl.col("nazev") == "Pomáda")

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/pomada-grease/16343""",3,10,0.3,"""Pomáda"""


In [52]:
pomer_full.filter(pl.col("nazev") == "Senzační prázdniny")

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/senzacni-prazdniny-les-grandes-vacances/18651""",5,3,1.666667,"""Senzační prázdniny"""


In [53]:
pomer_full.filter(pl.col("nazev") == "Bláznivá dovolená")

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/blazniva-dovolena-vacation/129004""",0,5,0.0,"""Bláznivá dovolená"""
"""film/blazniva-dovolena-vacation/4246""",2,2,1.0,"""Bláznivá dovolená"""


In [54]:
pomer_full.filter(pl.col("nazev") == "Slovácko sa nesúdí")

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""serial/slovacko-sa-nesudi/33944""",167,83,2.012048,"""Slovácko sa nesúdí"""


In [55]:
df.filter(pl.col("nazev") == "Slovácko sa nesúdí")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2011-03-17 11:10:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",3,11,2011,"""serial"""
"""ČT1""",2011-03-24 11:10:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",3,12,2011,"""serial"""
"""ČT1""",2011-03-31 11:10:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",3,13,2011,"""serial"""
"""ČT1""",2011-04-07 11:10:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",4,14,2011,"""serial"""
"""ČT1""",2011-04-14 11:10:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",4,15,2011,"""serial"""
"""ČT1""",2011-04-21 11:20:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",4,16,2011,"""serial"""
"""ČT1""",2011-04-28 11:10:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",4,17,2011,"""serial"""
"""ČT1""",2011-05-05 11:10:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",5,18,2011,"""serial"""
"""ČT1""",2011-05-12 11:05:00,"""Slovácko sa nesúdí""","""16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""Další díl/opakování:16.08.2024 v 14:05 (ČT1)17.08.2024 v 09:25 (ČT1)23.08.2024 v 14:25 (ČT1)""","""serial/slovacko-sa-nesudi/33944""",5,19,2011,"""serial"""


In [56]:
df.filter(pl.col("nazev") == "Bez ženské a bez tabáku").select(pl.col("kanal")).unique()

kanal
cat
"""ČT1"""


In [57]:
pomer_full.filter(pl.col("nazev").str.contains("Četník "))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/cetnik-v-new-yorku-le-gendarme-a-new-york/5617""",3,13,0.230769,"""Četník v New Yorku"""
"""film/cetnik-ve-vysluzbe-le-gendarme-en-balade/5618""",3,13,0.230769,"""Četník ve výslužbě"""
"""film/cetnik-a-cetnice-le-gendarme-et-les-gendarmettes/5613""",3,13,0.230769,"""Četník a četnice"""
"""film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614""",3,14,0.214286,"""Četník a mimozemšťané"""
"""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",1,12,0.083333,"""Četník ze St. Tropez"""
"""film/cetnik-se-zeni-le-gendarme-se-marie/5615""",3,13,0.230769,"""Četník se žení"""


In [58]:
pomer_full.filter(pl.col("nazev").str.contains("Hříšný tanec"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/hrisny-tanec-dirty-dancing/8788""",5,14,0.357143,"""Hříšný tanec"""
"""film/hrisny-tanec-2-dirty-dancing-havana-nights/37412""",0,6,0.0,"""Hříšný tanec 2"""


In [59]:
pomer_full.filter(pl.col("nazev").str.contains("Tajemství proutě"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""serial/tajemstvi-prouteneho-kosiku/29012""",39,26,1.5,"""Tajemství proutěného košíku"""


In [60]:
pomer_full.filter(pl.col("nazev").str.contains("z městečka"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/my-holky-z-mestecka/41510""",40,null,null,"""My holky z městečka (?) [TV minisérie]"""


In [61]:
pomer_full.filter(pl.col("nazev").str.contains("Slunce, seno"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/slunce-seno-a-par-facek/19067""",9,25,0.36,"""Slunce, seno a pár facek"""
"""film/slunce-seno-helena/69951""",0,1,0.0,"""Slunce, seno, Helena"""
"""film/slunce-seno-jahody/19069""",9,25,0.36,"""Slunce, seno, jahody"""
"""film/slunce-seno-erotika/19068""",9,25,0.36,"""Slunce, seno, erotika"""
"""film/slunce-seno-30-let/137939""",1,null,null,"""Slunce, seno, 30 let!"""


In [62]:
pomer_full.filter(pl.col("nazev").str.contains("Pomáda"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/pomada-2-grease-2/16344""",1,1,1.0,"""Pomáda 2"""
"""film/pomada-grease/16343""",3,10,0.3,"""Pomáda"""


In [63]:
pomer_full.filter(pl.col("nazev").str.contains("Turner a H"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/turner-a-hooch-turner-hooch/21494""",5,3,1.666667,"""Turner a Hooch"""


In [64]:
df.filter(pl.col("nazev").str.contains("Bez ženské a bez tabáku"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2008-11-06 09:05:00,"""Bez ženské a bez tabáku (?) [TV seriál]""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",11,45,2008,"""serial"""
"""ČT1""",2008-11-13 09:00:00,"""Bez ženské a bez tabáku (?) [TV seriál]""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",11,46,2008,"""serial"""
"""ČT1""",2008-11-20 09:00:00,"""Bez ženské a bez tabáku (?) [TV seriál]""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",11,47,2008,"""serial"""
"""ČT1""",2008-11-27 09:00:00,"""Bez ženské a bez tabáku (?) [TV seriál]""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",11,48,2008,"""serial"""
"""ČT1""",2012-06-12 03:40:00,"""Bez ženské a bez tabáku""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",6,24,2012,"""serial"""
"""ČT1""",2012-06-13 02:55:00,"""Bez ženské a bez tabáku""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",6,24,2012,"""serial"""
"""ČT1""",2012-06-14 03:40:00,"""Bez ženské a bez tabáku""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",6,24,2012,"""serial"""
"""ČT1""",2012-06-15 03:50:00,"""Bez ženské a bez tabáku""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",6,24,2012,"""serial"""
"""ČT1""",2014-05-17 07:35:00,"""Bez ženské a bez tabáku""","""30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""Další díl/opakování:30.08.2024 v 15:00 (ČT1)31.08.2024 v 08:35 (ČT1)06.09.2024 v 14:00 (ČT1)""","""serial/bez-zenske-a-bez-tabaku/41001""",5,20,2014,"""serial"""


In [65]:
pomer_full.filter(pl.col("nazev").str.contains("Stav ztroskotání"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/stav-ztroskotani/24691""",8,5,1.6,"""Stav ztroskotání"""


In [66]:
df.filter(pl.col("nazev").str.contains("Tajemství proutě"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2008-02-09 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02.2022 v 00:25 (ČT 3)…","""Další díl/opakování:13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02…","""serial/tajemstvi-prouteneho-kosiku/29012""",2,6,2008,"""serial"""
"""ČT1""",2008-02-16 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02.2022 v 00:25 (ČT 3)…","""Další díl/opakování:13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02…","""serial/tajemstvi-prouteneho-kosiku/29012""",2,7,2008,"""serial"""
"""ČT1""",2008-02-23 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02.2022 v 00:25 (ČT 3)…","""Další díl/opakování:13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02…","""serial/tajemstvi-prouteneho-kosiku/29012""",2,8,2008,"""serial"""
"""ČT1""",2008-03-01 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02.2022 v 00:25 (ČT 3)…","""Další díl/opakování:13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02…","""serial/tajemstvi-prouteneho-kosiku/29012""",3,9,2008,"""serial"""
"""ČT1""",2008-03-08 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02.2022 v 00:25 (ČT 3)…","""Další díl/opakování:13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02…","""serial/tajemstvi-prouteneho-kosiku/29012""",3,10,2008,"""serial"""
"""ČT1""",2008-03-15 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02.2022 v 00:25 (ČT 3)…","""Další díl/opakování:13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02…","""serial/tajemstvi-prouteneho-kosiku/29012""",3,11,2008,"""serial"""
"""ČT1""",2008-03-22 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02.2022 v 00:25 (ČT 3)…","""Další díl/opakování:13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02…","""serial/tajemstvi-prouteneho-kosiku/29012""",3,12,2008,"""serial"""
"""ČT1""",2008-03-29 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02.2022 v 00:25 (ČT 3)…","""Další díl/opakování:13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:15 (ČT 3)18.02…","""serial/tajemstvi-prouteneho-kosiku/29012""",3,13,2008,"""serial"""
"""ČT1""",2008-04-05 18:10:00,"""Tajemství proutěného košíku (?) [TV seriál]""","""13.02.2022 v 15:45 (ČT 3)14.02.2022 v 16:15 (ČT 3)15.02.2022 v 16:15 (ČT 3)16.02.2022 v 16:15 (ČT 3)17.02.2022 v 16:

In [67]:
pomer_full.filter(pl.col("v_lete") > 5).filter(pl.col("pomer") > 2).filter(pl.col("url").str.contains("film/"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""film/jdi-na-to-nati-con-la-camicia/9428""",8,1,8.0,"""Jdi na to"""
"""film/vinnetou-winnetou-i/22529""",20,2,10.0,"""Vinnetou"""
"""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",8,3,2.666667,"""Jestli se rozzlobíme, budeme zlí"""
"""film/my-little-pony/155069""",44,21,2.095238,"""My Little Pony"""
"""film/5-deti-a-to-five-children-and-it/41275""",12,3,4.0,"""5 dětí a To (?)"""
"""film/anynka-a-cert/36301""",11,1,11.0,"""Anynka a čert"""
"""film/bidnici-les-miserables/38283""",13,4,3.25,"""Bídníci"""
"""film/bourlive-vino/4519""",12,3,4.0,"""Bouřlivé víno"""
"""film/indiana-jones-a-chram-zkazy-indiana-jones-and-the-temple-of-doom/9107""",13,5,2.6,"""Indiana Jones a Chrám zkázy"""


In [68]:
pomer_full.filter(pl.col("nazev").str.contains("Slovácko sa"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""serial/slovacko-sa-nesudi/33944""",167,83,2.012048,"""Slovácko sa nesúdí"""


In [69]:
pomer_full.filter(pl.col("nazev").str.contains("bez tabáku"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""serial/bez-zenske-a-bez-tabaku/41001""",49,13,3.769231,"""Bez ženské a bez tabáku"""


In [70]:
df.filter(pl.col("nazev").str.contains("Četník ze"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2008-01-09 20:00:00,"""Četník ze Saint Tropez (?)""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025 v 22:45 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",1,2,2008,"""film"""
"""NOVA""",2009-03-01 15:05:00,"""Četník ze Saint Tropez (?)""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025 v 22:45 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",3,9,2009,"""film"""
"""NOVA""",2009-12-26 15:30:00,"""Četník ze St. Tropez""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025 v 22:45 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",12,52,2009,"""film"""
"""NOVA""",2010-10-02 16:30:00,"""Četník ze St. Tropez""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025 v 22:45 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",10,39,2010,"""film"""
"""NOVA""",2012-01-02 17:45:00,"""Četník ze St. Tropez""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025 v 22:45 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",1,1,2012,"""film"""
"""Nova Cinema""",2012-01-03 14:25:00,"""Četník ze St. Tropez""","""13.01.2024 v 22:35 (ČT1)18.01.2024 v 14:35 (ČT1)""","""Další díl/opakování:13.01.2024 v 22:35 (ČT1)18.01.2024 v 14:35 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",1,1,2012,"""film"""
"""NOVA""",2013-01-02 17:40:00,"""Četník ze St. Tropez""","""08.02.2025 v 22:45 (ČT1)""","""Další díl/opakování:08.02.2025 v 22:45 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",1,1,2013,"""film"""
"""Nova Cinema""",2013-01-03 15:40:00,"""Četník ze St. Tropez""","""13.01.2024 v 22:35 (ČT1)18.01.2024 v 14:35 (ČT1)""","""Další díl/opakování:13.01.2024 v 22:35 (ČT1)18.01.2024 v 14:35 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",1,1,2013,"""film"""
"""TV Barrandov""",2013-04-12 20:05:00,"""Četník ze Saint Tropez""","""13.01.2024 v 22:35 (ČT1)18.01.2024 v 14:35 (ČT1)""","""Další díl/opakování:13.01.2024 v 22:35 (ČT1)18.01.2024 v 14:35 (ČT1)""","""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",4,15,2013,"""film"""


In [71]:
df.filter(pl.col("kanal") == "ČT1").filter(pl.col("datum").dt.month().is_in([7,8])).filter(pl.col("nazev").str.contains("Chalupáři")).group_by(pl.col('datum').dt.year()).len().sort(by='datum')

datum,len
i32,u32
2012,18
2013,19
2017,9
2018,9
2019,11
2020,6
2021,11
2022,22
2023,22


In [72]:
df.filter(pl.col("nazev").str.contains("Chalupáři"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2008-01-20 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",1,3,2008,"""serial"""
"""ČT1""",2008-01-27 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",1,4,2008,"""serial"""
"""ČT1""",2008-02-03 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",2,5,2008,"""serial"""
"""ČT1""",2008-02-10 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",2,6,2008,"""serial"""
"""ČT1""",2008-02-17 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",2,7,2008,"""serial"""
"""ČT1""",2008-02-24 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",2,8,2008,"""serial"""
"""ČT1""",2008-03-02 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",3,9,2008,"""serial"""
"""ČT1""",2008-03-09 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",3,10,2008,"""serial"""
"""ČT1""",2008-03-16 11:00:00,"""Chalupáři (?) [TV seriál]""","""25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""Další díl/opakování:25.08.2024 v 08:15 (ČT1)01.09.2024 v 07:55 (ČT1)""","""serial/chalupari/28527""",3,11,2008,"""serial"""


In [73]:
df.filter(pl.col("nazev").str.contains("Ptáci v trní"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Prima""",2010-06-27 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",6,25,2010,"""film"""
"""Prima""",2010-07-04 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",7,26,2010,"""film"""
"""Prima""",2010-07-11 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",7,27,2010,"""film"""
"""Prima""",2010-07-18 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",7,28,2010,"""film"""
"""Prima""",2010-07-25 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",7,29,2010,"""film"""
"""Prima""",2010-08-01 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",8,30,2010,"""film"""
"""Prima""",2011-07-03 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",7,26,2011,"""film"""
"""Prima""",2011-07-10 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",7,27,2011,"""film"""
"""Prima""",2011-07-17 20:00:00,"""Ptáci v trní""","""31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""Další díl/opakování:31.03.2018 v 10:15 (Doma)01.04.2018 v 10:10 (Doma)02.04.2018 v 10:00 (Doma)""","""film/ptaci-v-trni-the-thorn-birds/17544""",7,28,2011,"""film"""


In [74]:
df.filter(pl.col("nazev").str.contains("Jestli se rozzlobíme, budeme zlí"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2007-07-14 17:40:00,"""Jestli se rozzlobíme, budeme zlí (?)""","""22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""Další díl/opakování:22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",7,28,2007,"""film"""
"""NOVA""",2008-09-21 09:25:00,"""Jestli se rozzlobíme, budeme zlí (?)""","""22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""Další díl/opakování:22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",9,38,2008,"""film"""
"""NOVA""",2009-09-20 08:50:00,"""Jestli se rozzlobíme, budeme zlí (?)""","""22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""Další díl/opakování:22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",9,38,2009,"""film"""
"""NOVA""",2010-07-04 10:00:00,"""Jestli se rozzlobíme, budeme zlí""","""22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""Další díl/opakování:22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",7,26,2010,"""film"""
"""NOVA""",2010-07-05 01:20:00,"""Jestli se rozzlobíme, budeme zlí""","""22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""Další díl/opakování:22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",7,27,2010,"""film"""
"""Nova Cinema""",2012-01-03 20:00:00,"""Jestli se rozzlobíme, budeme zlí""","""05.11.2024 v 18:45 (Film+)08.11.2024 v 16:55 (Film+)10.11.2024 v 12:55 (Film+)""","""Další díl/opakování:05.11.2024 v 18:45 (Film+)08.11.2024 v 16:55 (Film+)10.11.2024 v 12:55 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",1,1,2012,"""film"""
"""NOVA""",2012-08-22 08:55:00,"""Jestli se rozzlobíme, budeme zlí""","""22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""Další díl/opakování:22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",8,34,2012,"""film"""
"""Nova Cinema""",2012-08-22 15:00:00,"""Jestli se rozzlobíme, budeme zlí""","""05.11.2024 v 18:45 (Film+)08.11.2024 v 16:55 (Film+)10.11.2024 v 12:55 (Film+)""","""Další díl/opakování:05.11.2024 v 18:45 (Film+)08.11.2024 v 16:55 (Film+)10.11.2024 v 12:55 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",8,34,2012,"""film"""
"""NOVA""",2013-08-18 09:20:00,"""Jestli se rozzlobíme, budeme zlí""","""22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""Další díl/opakování:22.03.2025 v 13:00 (Film+)25.03.2025 v 12:40 (Film+)28.03.2025 v 15:05 (Film+)""","""film/jestli-se-rozzlobime-budeme-zli-altrimenti-ci-arrabbiamo/9615""",8,33,2013,"""film"""


In [75]:
pomer_full.filter(pl.col("v_lete") > 15).filter(pl.col("pomer") > 2).filter(pl.col("url").str.contains("serial/"))

url,v_lete,v_zime,pomer,nazev
str,u32,u32,f64,str
"""serial/neuveritelna-dobrodruzstvi-rudly-a-koumaka-pinky-and-the-brain/54970""",44,21,2.095238,"""Neuvěřitelná dobrodružství Rudly a Koumáka"""
"""serial/pan-tau-2-serie-13-dil-pan-tau-odchazi/37966""",61,10,6.1,"""Pan Tau"""
"""serial/kamenak/163648""",61,25,2.44,"""Kameňák"""
"""serial/ten-kdo-te-chrani-the-guardian/33947""",34,13,2.615385,"""Ten, kdo tě chrání"""
"""serial/dobra-voda/40349""",77,28,2.75,"""Dobrá Voda"""
"""serial/etiketa/39306""",720,271,2.656827,"""Etiketa"""
"""serial/hospoda/8564""",461,108,4.268519,"""Hospoda"""
"""serial/slovacko-sa-nesudi/33944""",167,83,2.012048,"""Slovácko sa nesúdí"""
"""serial/zachranari/39506""",68,6,11.333333,"""Záchranáři"""


## Zmizelé - 2nd take

In [77]:
df.select(pl.col("datum").min())

datum
datetime[μs]
2007-01-01 10:05:00


In [78]:
porovnani = df.filter(
    pl.col("kanal").is_in(zajima_nas)
).filter(
    pl.col("url").str.contains("film/")
).filter(
    pl.col("datum").dt.year().is_between(2007,2011)
).filter(
    pl.col("datum").dt.month().is_between(7,8)
).group_by(
    "url"
).len(
).sort(
    by="len",descending=True
).rename(
    {'len':'kdysi'}
).join(
    df.filter(
        pl.col("kanal").is_in(zajima_nas)
    ).filter(
        pl.col("url").str.contains("film/")
    ).filter(
        pl.col("datum").dt.year().is_between(2020,2024)
    ).filter(
        pl.col("datum").dt.month().is_between(7,8)
    ).group_by(
        "url"
    ).len(
    ).rename(
        {'len':'nyni'}
    ),
    how="full", on="url"
)

In [79]:
porovnani2 = df.filter(
    pl.col("kanal").is_in(zajima_nas)
).filter(
    pl.col("url").str.contains("film/")
).filter(
    pl.col("datum").dt.year().is_between(2007,2013)
).filter(
    pl.col("datum").dt.month().is_between(6,9)
).group_by(
    "nazev"
).agg(
    pl.col('datum').dt.year().unique().len()
).rename(
    {'datum':'kdysi'}
).join(
    df.filter(
        pl.col("kanal").is_in(zajima_nas)
    ).filter(
        pl.col("url").str.contains("film/")
    ).filter(
        pl.col("datum").dt.year().is_between(2018,2024)
    ).filter(
        pl.col("datum").dt.month().is_between(6,9)
    ).group_by(
        "nazev"
    ).agg(
        pl.col('datum').dt.year().unique().len()
    ).rename(
        {'datum':'nyni'}
    ),
    on='nazev',how='full'
    
)

In [80]:
porovnani2.filter(pl.col('nazev').str.contains("Hříšn"))

nazev,kdysi,nazev_right,nyni
str,u32,str,u32
"""Hříšní lidé města brněnského""",2,"""Hříšní lidé města brněnského""",4
"""Hříšníci a svatí""",1,null,null
"""Hříšné noci""",1,null,null
"""Hříšní lidé města brněnského (?) [TV minisérie]""",1,null,null
"""Hříšný tanec""",1,"""Hříšný tanec""",3
"""Hříšný tanec (?)""",1,null,null


In [81]:
porovnani2.filter(pl.col('kdysi') > 5).filter(pl.col('nyni').is_null() | (pl.col('nyni') < 3))

nazev,kdysi,nazev_right,nyni
str,u32,str,u32
"""DO-RE-MI""",7,"""DO-RE-MI""",2
"""Volejte Věštce""",6,null,null
"""Uvolněte se, prosím""",7,"""Uvolněte se, prosím""",1
"""Autosalon""",7,"""Autosalon""",2
"""Nikdo není dokonalý""",7,null,null
"""Bludiště""",6,null,null
"""Auto Moto Revue""",6,null,null
"""Malý televizní kabaret""",6,null,null
"""Terra musica""",6,null,null


In [82]:
porovnani.filter(pl.col('nyni').is_null()).sort(by='kdysi',descending=True)

url,kdysi,url_right,nyni
str,u32,str,u32
"""film/minuty-regionu/83029""",255,null,null
"""film/volejte-vestce/83154""",247,null,null
"""film/party-s-kucharem/89955""",153,null,null
"""film/5-proti-5/56231""",134,null,null
"""film/fakta-barbory-tacheci/84750""",126,null,null
"""film/cerne-ovce/73750""",118,null,null
"""film/vip-zpravy/82821""",115,null,null
"""film/acko/75221""",98,null,null
"""film/na-stope/82815""",95,null,null


## Neletní věci

In [84]:
df.filter(pl.col('nazev').str.contains('Pomád'))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2007-08-19 12:30:00,"""Pomáda (?)""","""13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""Další díl/opakování:13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""film/pomada-grease/16343""",8,33,2007,"""film"""
"""NOVA""",2007-08-21 09:25:00,"""Pomáda 2 (?)""","""29.12.2010 v 20:00 (Nova Cinema)""","""Další díl/opakování:29.12.2010 v 20:00 (Nova Cinema)""","""film/pomada-2-grease-2/16344""",8,34,2007,"""film"""
"""NOVA""",2009-02-08 11:50:00,"""Pomáda (?)""","""13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""Další díl/opakování:13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""film/pomada-grease/16343""",2,6,2009,"""film"""
"""NOVA""",2010-02-13 09:50:00,"""Pomáda""","""13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""Další díl/opakování:13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""film/pomada-grease/16343""",2,6,2010,"""film"""
"""NOVA""",2010-02-18 09:30:00,"""Pomáda 2""","""29.12.2010 v 20:00 (Nova Cinema)""","""Další díl/opakování:29.12.2010 v 20:00 (Nova Cinema)""","""film/pomada-2-grease-2/16344""",2,7,2010,"""film"""
"""Nova Cinema""",2010-12-29 20:00:00,"""Pomáda 2""",null,null,"""film/pomada-2-grease-2/16344""",12,52,2010,"""film"""
"""NOVA""",2011-05-17 09:35:00,"""Pomáda""","""13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""Další díl/opakování:13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""film/pomada-grease/16343""",5,20,2011,"""film"""
"""Nova Cinema""",2012-06-23 20:00:00,"""Pomáda""","""13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""Další díl/opakování:13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""film/pomada-grease/16343""",6,25,2012,"""film"""
"""Nova Cinema""",2012-06-24 09:10:00,"""Pomáda""","""13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""Další díl/opakování:13.10.2024 v 20:00 (Prima MAX)15.10.2024 v 00:00 (Prima MAX)20.10.2024 v 16:30 (Prima)""","""film/pomada-grease/16343""",6,25,2012,"""film"""


In [85]:
nazvy = df.group_by('url').agg(pl.col('nazev').mode()).explode('nazev')
nazvy.sample(3)

url,nazev
str,str
"""film/loft/69678""","""Loft"""
"""film/pozor-na-kozoroha/41104""","""Pozor na kozoroha"""
"""film/kdo-je-tvuj-tata-who-s-your-daddy/171373""","""Kdo je tvůj táta"""


In [86]:
vyber_neletni = [
    'film/slunce-seno-jahody/19069',
    'film/slunce-seno-a-par-facek/19067',
    'film/slunce-seno-erotika/19068',
    'film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619',
    'film/cetnik-v-new-yorku-le-gendarme-a-new-york/5617',
    'film/cetnik-se-zeni-le-gendarme-se-marie/5615',
    'film/cetnik-ve-vysluzbe-le-gendarme-en-balade/5618',
    'film/cetnik-a-mimozemstane-le-gendarme-et-les-extraterrestres/5614',
    'film/cetnik-a-cetnice-le-gendarme-et-les-gendarmettes/5613',
    'film/leto-s-kovbojem/11391',
    'film/hrisny-tanec-dirty-dancing/8788',
    'film/fantomas-fantomas/7562',
    'film/fantomas-se-zlobi-fantomas-se-dechane/7564',
    'film/fantomas-kontra-scotland-yard-fantomas-contre-scotland-yard/7563',
    'film/horecka-sobotni-noci-saturday-night-fever/8502',
    'film/pomada-grease/16343'
]

In [87]:
graf_neletni = df.filter(
    pl.col("rok").is_between(2015,2024)
).filter(
    pl.col('url').is_in(vyber_neletni)
).filter(
    ~pl.col('url').is_in(nechceme)
).filter(
    pl.col("kanal").is_in(zajima_nas)
).group_by(
    ['url','tyden']
).len(
).join(
    nazvy, on='url', how='left'
)

graf_neletni

url,tyden,len,nazev
str,i8,u32,str
"""film/cetnik-a-cetnice-le-gendarme-et-les-gendarmettes/5613""",50,3,"""Četník a četnice"""
"""film/cetnik-v-new-yorku-le-gendarme-a-new-york/5617""",29,1,"""Četník v New Yorku"""
"""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",3,1,"""Četník ze St. Tropez"""
"""film/leto-s-kovbojem/11391""",21,3,"""Léto s kovbojem"""
"""film/slunce-seno-erotika/19068""",7,1,"""Slunce, seno, erotika"""
"""film/slunce-seno-jahody/19069""",27,1,"""Slunce, seno, jahody"""
"""film/slunce-seno-erotika/19068""",20,3,"""Slunce, seno, erotika"""
"""film/cetnik-se-zeni-le-gendarme-se-marie/5615""",4,1,"""Četník se žení"""
"""film/cetnik-ze-saint-tropez-le-gendarme-de-saint-tropez/5619""",28,1,"""Četník ze St. Tropez"""


In [88]:
graf_neletni.select(pl.col("nazev")).unique()

nazev
str
"""Četník se žení"""
"""Četník v New Yorku"""
"""Fantomas kontra Scotland Yard"""
"""Fantomas"""
"""Hříšný tanec"""
"""Slunce, seno a pár facek"""
"""Léto s kovbojem"""
"""Fantomas se zlobí"""
"""Pomáda"""


In [89]:
razeni_neletni = [
    "Slunce, seno, jahody",
    "Slunce, seno a pár facek",
    "Slunce, seno, erotika",
    "Fantomas",
    "Fantomas se zlobí",
    "Fantomas kontra Scotland Yard",
    "Senzační prázdniny",
    "Hříšný tanec",
    "Pomáda",
    "Horečka sobotní noci",
    "Léto s kovbojem",
    "Četník ze St. Tropez",
    "Četník v New Yorku",
    "Četník se žení",
    "Četník ve výslužbě",
    "Četník a mimozemšťané",
    "Četník a četnice",


]

In [183]:
obsah_neletni = alt.Chart(
    graf_neletni,
    title=alt.Title(
        'Zdánlivě letní filmy',
        subtitle=[
            'Snímky s letní tematikou nebo označované za letní klasiky v komentářích na ČSFD, které však na ČT1, ČT2,',
            'Nově a Primě nepatří jednoznačně do prázdninových schémat. Opět jde o přehled repríz v letech 2015 až 2024.'
        ],
        subtitleFontSize=9.3
    )
).mark_circle(
    color='#D15A41'
).encode(
    alt.X('tyden:Q', axis=alt.Axis(
        title="týden v roce", tickColor='#F1D1BB', tickOpacity=0.6, tickCount=5, tickWidth = 0.5,
        labelExpr="datum.label + '.'"
    ), scale=alt.Scale(
        domainMin = 1,
        domainMax = 52,
        padding=0,        # Remove extra padding
        nice=False        # Prevent extending to "nice" numbers
    )),
    alt.Y('nazev:N', sort=razeni_neletni, axis=alt.Axis(title=None, tickColor='#F1D1BB', tickOpacity=0.6, tickWidth = 0.5)),
    alt.Size('len:Q', scale=alt.Scale(range=[25,100]), legend=None),
).properties(
    width="container",
)

popisek_neletni = alt.Chart(
    pl.DataFrame({'tyden':38})
).mark_text(
    font='Asap',
    color='#292829',
    fontSize=8,
    text=['← týdny,','do kterých','zasáhly letní','prázdniny'],
    align='left',
    dy=-115
).encode(alt.X("tyden:Q"))

finale_neletni = alt.layer(terminy, popisek_neletni, obsah_neletni).configure_view(
    stroke='transparent'
).configure_axis(grid=True, gridOpacity=0.6, gridWidth = 0.5, gridColor='#F1D1BB').properties(
    width=325,
    height=250
)

finale_neletni

alt.LayerChart(...)

In [185]:
me_to_neurazi(graf=finale_neletni, kredity="vizualizace: iROZHLAS.cz | televizní programy: fdb.cz", slozka_na_serveru="grafy-tv", soubor='neletni-program')

<figure>
    <a href="https://data.irozhlas.cz/grafy-tv/neletni-program.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy-tv/neletni-program.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>
